# Cleaning the Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sqlite3
from zipfile import ZipFile # for unzipping the db.zip file
pd.set_option('display.max_columns', None) # show all columns when printing out dataframe

## "The Numbers" Movie Budgets

In [3]:
tn_budgets_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
tn_budgets_df.shape

(5782, 6)

In [4]:
# remove commas and $ from the strings of numeric columns
tn_budgets_df['production_budget'] = tn_budgets_df['production_budget'].str.replace('$', '').str.replace(',', '')
tn_budgets_df['domestic_gross'] = tn_budgets_df['domestic_gross'].str.replace('$', '').str.replace(',', '')
tn_budgets_df['worldwide_gross'] = tn_budgets_df['worldwide_gross'].str.replace('$', '').str.replace(',', '')

In [5]:
# convert numeric columns to ints
tn_budgets_df['production_budget'] = tn_budgets_df['production_budget'].astype(int)
tn_budgets_df['domestic_gross'] = tn_budgets_df['domestic_gross'].astype(int)
tn_budgets_df['worldwide_gross'] = tn_budgets_df['worldwide_gross'].astype(int)

In [6]:
# create a column for the profit of every movie
tn_budgets_df['gross_profit'] = 0
tn_budgets_df['gross_profit'] = tn_budgets_df['worldwide_gross'] - tn_budgets_df['production_budget']

In [7]:
# divide all budget columns by 1M to make numbers easier to work with
tn_budgets_df['production_budget'] = round(tn_budgets_df['production_budget'] / 1000000, 3)
tn_budgets_df['domestic_gross'] = round(tn_budgets_df['domestic_gross'] / 1000000, 3) 
tn_budgets_df['worldwide_gross'] = round(tn_budgets_df['worldwide_gross'] / 1000000, 3) 
tn_budgets_df['gross_profit'] = round(tn_budgets_df['gross_profit'] / 1000000, 3)

In [8]:
# access to the specific year a movie was made in
tn_budgets_df['movie_year'] = tn_budgets_df['release_date'].map(lambda x: x[-4:]).astype(int)

In [9]:
# create a datetime column
tn_budgets_df["date_time"] = pd.to_datetime(tn_budgets_df['release_date'])

In [10]:
# create an ROI column
tn_budgets_df['roi'] = tn_budgets_df['gross_profit'] / tn_budgets_df['production_budget']

In [11]:
tn_budgets_df['two_x'] = 0
tn_budgets_df['two_x'] = tn_budgets_df['roi'].map(lambda x: 1 if x >= 1 else 0)

In [12]:
# removing data from the table for complete losses, seems to be impacted mostly by streamers or non-released movies
tn_budgets_df = tn_budgets_df.loc[tn_budgets_df['roi'] != -1]

In [13]:
# Creating quartile groups based on production budget
tn_budgets_df['quartile'] = 0
tn_budgets_df["quartile"] = pd.qcut(tn_budgets_df["production_budget"], q=4, labels=["Bottom 25", "25-50", "50-75", "Top 25"])

Actual creation of the new CSV file

In [14]:
# tn_budgets_df.to_csv("cleaned_budget_data.csv", index=False)

In [15]:
tn_budgets_df.describe()

,id,production_budget,domestic_gross,worldwide_gross,gross_profit,movie_year,date_time,roi,two_x
count,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410,5410.000000,5410.000000
mean,50.234750,33.338222,44.752602,97.778280,64.440058,2003.592421,2004-02-17 20:22:16.192236672,4.133304,0.484473
min,1.000000,0.001000,0.000000,0.001000,-200.238000,1915.000000,1915-02-08 00:00:00,-0.999889,0.000000
25%,25.000000,6.000000,3.209500,7.018250,-1.874750,1999.000000,1999-12-10 00:00:00,-0.296339,0.000000
50%,50.000000,19.000000,20.347000,33.395000,11.932500,2006.000000,2006-08-11 00:00:00,0.888318,0.000000
75%,75.000000,42.000000,55.837500,104.500750,67.424500,2012.000000,2012-05-14 18:00:00,2.974420,1.000000
max,100.000000,425.000000,936.662000,2776.345000,2351.345000,2019.000000,2019-06-21 00:00:00,1799.000000,1.000000
std,28.762457,42.495577,69.628779,178.917092,149.914269,12.550054,NaN,30.517830,0.499805
